In [1]:
#!/usr/bin/env python3
'''
Based on the work done in 
On-Manifold Preintegration for Real-Time Visual-Inertial Odometry (Carlone et al.),
and Vision-Based Control of Multirotor Systems (yet to be published)
'''

import numpy as np
from scipy.linalg import expm, logm
import matplotlib.pyplot as plt
from copy import deepcopy

def skew(v):
    return np.array([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])

In [2]:
class IMU:
    def __init__(self):
        self.ba = np.zeros(3) #Leave as zeros for now
        self.bw = np.zeros(3)
        self.covar_a = np.diag([0.00561, 0.00561, 0.00561]) #from quadsim
        self.covar_w = np.diag([0.00025, 0.00025, 0.00025]) #from quadsim
    
    def getMeas(self, a_nom, w_nom):
        noise_a = np.random.multivariate_normal(np.zeros(3), self.covar_a)
        noise_w = np.random.multivariate_normal(np.zeros(3), self.covar_w)
        a = a_nom + self.ba + noise_a
        w = w_nom + self.bw + noise_w
        
        return a, w

In [3]:
class State:
    def __init__(self, p=np.zeros(3), v=np.zeros(3), R=np.eye(3), w=np.zeros(3)):
        self.p_i = p
        self.v_i = v
        self.Ri_from_b = R
        self.w_b = w
    
    def __mul__(self, dt):
        self.p_i = self.p_i * dt
        self.v_i = self.v_i * dt
        self.Ri_from_b = self.Ri_from_b * dt
        self.w_b = self.w_b * dt #Note that this isn't alpha. It is still omega
        
        return self
    
    def __rmul__(self, val):
        return self * val
    
    def __add__(self, dx):
        self.p_i = self.p_i + dx.p_i
        self.v_i = self.v_i + dx.v_i
        self.Ri_from_b = self.Ri_from_b @ expm(skew(dx.w_b))
        
        return self
    
    def __sub__(self, state2):
        dp = (self.p_i - state2.p_i)
        dv = (self.v_i - state2.v_i)
        dR = self.Ri_from_b.T @ state2.Ri_from_b
        
        return State(dp, dv, dR)
    
    def normalize(self): #Graham Schmidt orthonormalization or rotation matrix
        x = self.Ri_from_b[:,0] / np.linalg.norm(self.Ri_from_b[:,0])
        y = skew(self.Ri_from_b[:,2]) @ x
        y = y / np.linalg.norm(y)
        z = skew(x) @ y
        z = z/np.linalg.norm(z)
        self.Ri_from_b = np.array([[*x], [*y], [*z]]).T
    
    def preIntegrate(self, u, dt):
        a = u.a_b
        w = u.w_b
        self.Ri_from_b = self.Ri_from_b @ expm(skew(w) * dt)
        self.v_i += self.Ri_from_b @ a * dt
        self.p_i += self.v_i * dt + (dt**2)/2 * self.Ri_from_b @ a
    
    def preIntegrateBeard(self, dx_prev, u, u_prev, dt):
        a = u.a_b
        w = u.w_b
        Rlp1_l = expm(-skew(w) * dt)
        self.Ri_from_b = Rlp1_l @ self.Ri_from_b
        
        vl_lp1 = Rlp1_l @ (dx_prev.v_i + dt * (dx_prev.Ri_from_b @ u_prev.a_b - a))
        self.v_i = Rlp1_l @ self.v_i + vl_lp1
        
        pl_lp1 = Rlp1_l @ (dx_prev.p_i + dt * dx_prev.v_i + dt**2/2 * (dx_prev.Ri_from_b@u_prev.a_b - a))
        self.p_i = Rlp1_l @ self.p_i + pl_lp1
        
        dx_prev.Ri_from_b = Rlp1_l
        dx_prev.v_i = vl_lp1
        dx_prev.p_i = pl_lp1
        return dx_prev

class Input:
    def __init__(self, a = np.zeros(3), w=np.zeros(3)):
        self.a_b = a
        self.w_b = w
    
    def update(self, a, w):
        self.a_b = a
        self.w_b = w

In [4]:
class Multirotor:
    def __init__(self, m, J):
        self.m = m
        self.J = J
        self.g = 9.81
        
        # Define states for the multirotor
        self.state = State()
    
    def dynamics(self, state, u):
        v_i = state.v_i
        Ri_from_b = state.Ri_from_b
        a_b = u.a_b
        w_b = u.w_b
        e3 = np.array([0, 0, 1])
        
        state_dot = State()
        state_dot.p_i = v_i
        state_dot.v_i = self.g * e3 + Ri_from_b @ a_b
        state_dot.Ri_from_b = Ri_from_b @ skew(w_b)
        state_dot.w_b = w_b #Not the derivative but this will be used in the integration
        
        return state_dot
    
    def update(self, u, dt):
#         k1 = self.dynamics(self.state, u)
#         k2 = self.dynamics(self.state + k1 * (dt/2), u)
#         k3 = self.dynamics(self.state + k2 * (dt/2), u)
#         k4 = self.dynamics(self.state + k3 * dt, u)
        
#         self.state = self.state + (dt/6) * (k1 + 2 * k2 + 2*k3 + k4)
        dx = self.dynamics(self.state, u)
        self.state = self.state + dt * dx
        return self.state


In [5]:
mass = 2.856
J = np.diag([.07, .08, .12])

quad = Multirotor(mass, J)
#Test to see if dynamics work
for i in range(100):
    a = np.array([1, 0, 1])
    w = np.array([.1, 0.0, .1])
    dt = .01
    u = Input(a, w)
    temp = quad.update(u, dt)
    if not np.linalg.det(temp.Ri_from_b) == 1.0:
        quad.state.normalize()
print(temp.p_i)
print(temp.v_i)
print(temp.Ri_from_b)
print(np.linalg.det(temp.Ri_from_b))

def getAccel(t):
    T = 5
    A = 0.1
    ax = A * np.cos(2 * np.pi/T * t)
    ay = A * np.sin(2 * np.pi/T * t)
    az = A * np.cos(2 * np.pi/T * t - np.pi/4)
    return np.array([ax, ay, az])

def getGyro(t):
    T = 5
    A = 0.1
    wx = A * np.cos(2 * np.pi/T * t)
    wy = A * np.sin(2 * np.pi/T * t)
    wz = A * np.sin(2 * np.pi/T * t - np.pi/4)
    return np.array([wx, wy, wz])

[4.95000000e-01 1.89029348e-17 5.35095000e+00]
[1.00000000e+00 7.23683266e-17 1.08100000e+01]
[[ 0.99500833 -0.099667    0.00499167]
 [ 0.099667    0.99001666 -0.099667  ]
 [ 0.00499167  0.099667    0.99500833]]
1.0


In [6]:
quad1 = Multirotor(mass, J) #No Imu preintegration
# quad2 = Multirotor(mass, J) #Dr. Beards Method
# quad3 = Multirotor(mass, J) #Carlone Paper Method
imu = IMU()

quad1_state = [deepcopy(quad1.state)]
print(quad1_state[0].p_i)
# quad2_state = [deepcopy(quad2.state)]
# quad3_state = [deepcopy(quad3.state)]

dt = 0.01
t = 0.0
num_steps = 200
kf_steps = 100

x_rel2 = State() #Relative Pose: Dr. Beards Method
dx_prev = State()
u_prev = Input()
x_rel3 = State() #Relative Pose: Carlone Paper
rel_pose_true = []
rel_pose2 = []
rel_pose3 = []
for i in range(1, num_steps):
    #Get IMU Measurement
    a_nom = getAccel(t)
    w_nom = getGyro(t)
    a, w = imu.getMeas(a_nom, w_nom)
    u = Input(a, w) #Have IMU return this. Fix later
    
    #Update dynamics
    state1 = quad1.update(u, dt)
    if not np.linalg.det(state1.Ri_from_b) == 1.0:
        quad1.state.normalize()
    quad1_state.append(deepcopy(quad1.state))
    
    #Integrate IMU
    #Carlone Method
    x_rel3.preIntegrate(u, dt)
    if not np.linalg.det(x_rel3.Ri_from_b) == 1.0:
        x_rel3.normalize()
    
    #Dr. Beard's method
    dx_prev = x_rel2.preIntegrateBeard(dx_prev, u, u_prev, dt)
    u_prev = deepcopy(u)
    
    if i%kf_steps == 0:
        rel_pose3.append(deepcopy(x_rel3))
        x_rel3 = State()
        
        rel_pose2.append(deepcopy(x_rel2))
        x_rel2 = State()
        
        rel_pose_true.append(quad1_state[i] - quad1_state[i-kf_steps])

[0. 0. 0.]


In [9]:
#Plot error between truth and imu_preintegration
#Need to verify stuff is in the right frame
print(rel_pose2[0].Ri_from_b)
print(rel_pose3[0].Ri_from_b) # Transposed from truth
print(rel_pose_true[0].Ri_from_b)
print(quad1_state[kf_steps].Ri_from_b.T) #Want this to be transposed so that is is rotation from former to current
print()
print()
print(rel_pose2[0].p_i) #Don't expect this one to match
print(rel_pose3[0].p_i) #I believe that this one should match truth though
print(rel_pose_true[0].p_i)
print(quad1_state[kf_steps].p_i)

[[ 0.99752372 -0.07021299 -0.00406986]
 [ 0.07027574  0.99278157  0.0971909 ]
 [-0.00278358 -0.09723624  0.99525744]]
[[ 0.99752372  0.07027574 -0.00278358]
 [-0.07021299  0.99278157 -0.09723624]
 [-0.00406986  0.0971909   0.99525744]]
[[ 0.99752372 -0.07021299 -0.00406986]
 [ 0.07027574  0.99278157  0.0971909 ]
 [-0.00278358 -0.09723624  0.99525744]]
[[ 0.99752372 -0.07021299 -0.00406986]
 [ 0.07027574  0.99278157  0.0971909 ]
 [-0.00278358 -0.09723624  0.99525744]]


[-0.00016914 -0.03260846 -0.02116558]
[ 0.04802908 -0.00426855  0.03132362]
[ 4.66500047e-02 -4.08185579e-03  4.88639438e+00]
[ 4.66500047e-02 -4.08185579e-03  4.88639438e+00]
